In [49]:
from z3 import *

import yaml
import pandas as pd

from dataclasses import dataclass
from pprint import pprint
from typing import Optional
from itertools import product

pd.set_option('display.max_rows', None)

In [50]:
MM_FILE = './assets/metamodels/doml_meta_v2.0.yaml'

# IM_FILE = './assets/doml/2.0/nginx-openstack_v2.0.yaml'
IM_FILE = './assets/doml/2.0/nginx-openstack_v2.0_wrong_vm_iface.yaml'

UB_ELEMS = 0

In [51]:
with open(MM_FILE, 'r') as mm_file:
    mm = yaml.safe_load(mm_file)

with open(IM_FILE, 'r') as im_file:
    im = yaml.safe_load(im_file)

We need to create first the elements by parsing the metamodel

In [52]:
classes = { f'{cat_k}_{elem_k}' : elem_v
    for cat_k, cat_v in mm.items()
    for elem_k, elem_v in cat_v.items() 
}

def merge_superclass(elem):
    sc = classes.get(elem.get('superclass'))
    if sc: # merge superclass
        sc_merged = merge_superclass(sc)
        return {
            'attributes': elem.get('attributes', {}) | sc_merged.get('attributes', {}),
            'associations': elem.get('associations', {}) | sc_merged.get('associations', {})
        }
    else: # return the elem
        return elem

# all the elements with all the inherited attributes and associations
merged_classes = {
    elem_k : merge_superclass(elem_v)
    for elem_k, elem_v in classes.items()
}

In [53]:

@dataclass
class Class:
    name: str
    attributes: dict[str, dict]
    associations: dict[str, dict]
    ref: Optional[SortRef] = None

@dataclass
class Elem:
    id: str
    name: Optional[str]
    attributes: dict[str, dict]
    associations: dict[str, dict]
    eClass: Optional[Class] = None
    ref: Optional[SortRef] = None
    unbound: bool = False

@dataclass
class AssocRel:
    from_elem: Class
    to_elem: Class
    inverse_of: Optional[str]
    ref: Optional[FuncDeclRef] = None

@dataclass
class AttrRel:
    multiplicity: str
    type: str
    ref: Optional[FuncDeclRef] = None

# Convert ELEMS
CLASSES = {
    class_k : Class(
        class_k, 
        class_v.get('attributes', {}), 
        class_v.get('associations', {})
    )
    for class_k, class_v in merged_classes.items()
}

ELEMS = {
    elem_k : Elem(
        elem_v['id'],
        elem_v['name'],
        elem_v['attrs'],
        elem_v['assocs'],
        CLASSES[elem_v['class']],
    )
    for elem_k, elem_v in im.items()
}

# This also helps catching errors in class/assoc names
ASSOCS = {
    f'{class_k}::{assoc_k}' : AssocRel(class_v, CLASSES[assoc_v['class']], assoc_v.get('inverse_of', None))
    for class_k, class_v in CLASSES.items()
    for assoc_k, assoc_v in class_v.associations.items()
}

# Careful: I decided to default multiplicity to 0..1
ATTRS = {
    f'{class_k}::{attr_k}' : AttrRel(attr_v.get('multiplicity', '0..1'), attr_v['type'])
    for class_k, class_v in CLASSES.items()
    for attr_k, attr_v in class_v.attributes.items()
}

Visualization of all the Elems and Assocs

In [54]:
classes_df = pd.DataFrame.from_dict([
    {
        'name': name
    }
    for name, value in CLASSES.items()
])

classes_df

,name
0,commons_DOMLElement
1,commons_Property
2,commons_IProperty
3,commons_SProperty
4,commons_FProperty
5,commons_BProperty
6,commons_Configuration
7,commons_Deployment
8,application_ApplicationLayer
9,application_ApplicationComponent


In [55]:
assoc_df = pd.DataFrame.from_dict([
    {
        'name': name,
        'from': value.from_elem.name,
        'to': value.to_elem.name,
        'inverse': value.inverse_of
    }
    for name, value in ASSOCS.items()
])

assoc_df[['from', 'name', 'to', 'inverse']]

,from,name,to,inverse
0,commons_DOMLElement,commons_DOMLElement::annotations,commons_Property,None
1,commons_Property,commons_Property::reference,commons_DOMLElement,None
2,commons_IProperty,commons_IProperty::reference,commons_DOMLElement,None
3,commons_SProperty,commons_SProperty::reference,commons_DOMLElement,None
4,commons_FProperty,commons_FProperty::reference,commons_DOMLElement,None
5,commons_BProperty,commons_BProperty::reference,commons_DOMLElement,None
6,commons_Configuration,commons_Configuration::deployments,commons_Deployment,None
7,commons_Configuration,commons_Configuration::annotations,commons_Property,None
8,commons_Deployment,commons_Deployment::component,application_ApplicationComponent,None
9,commons_Deployment,commons_Deployment::node,infrastructure_InfrastructureElement,None


### Z3 Enum Definitions
Here we define the Sorts of Elements and Associations as Enums (as they are finite), and the relationship between two elements as a Function `AssocRel :: ElemSort, AssocSort, ElemSort -> BoolSort`, which tells us if two items are in a relationship (returns true) or not.

In [56]:
# Init Z3 solver
s = Solver()

def Iff(a, b):
    return a == b

In [57]:
ub_elems = { f'elem_ub_{i}' : Elem(
    f'elem_ub_{i}',
    f'Unbound Element #{i}',
    {},
    {},
    unbound=True
    ) 
    for i in range(UB_ELEMS)
}

In [58]:
class_sort, classes = EnumSort('Class', list(CLASSES.keys()))
# Add the Ref to each ELEM
for c in classes:
    CLASSES[str(c)].ref = c

elem_sort, elems = EnumSort('Elem', list((ELEMS | ub_elems).keys()))
for elem in elems:
    ELEMS[str(elem)].ref = elem

# Where assoc_sort is an EnumSort of all associations names...
assoc_sort, assocs = EnumSort('Assoc', list(ASSOCS.keys()))
# Add the Ref to each ASSOC
for assoc in assocs:
    ASSOCS[str(assoc)].ref = assoc

attr_sort, attrs = EnumSort('Attr', list(ATTRS.keys()))
# Add the Ref to each ATTR
for attr in attrs:
    ATTRS[str(attr)].ref = attr

In [59]:
# ElemClass(Elem) -> Class
elem_class_fn = Function('ElemClass', elem_sort, class_sort)

for _, elem in ELEMS.items():
    if not elem.unbound:
        user_friendly_name = f'({elem.name})' if elem.name else ''
        s.assert_and_track(elem_class_fn(elem.ref) == elem.eClass.ref, f'ElemClass {elem.id}{user_friendly_name} {elem.eClass.name}')

In [60]:
# AssocRel(Elem, Assoc, Elem) -> Bool
assoc_rel = Function('AssocRel', elem_sort, assoc_sort, elem_sort, BoolSort())

assoc_a = Const('assoc_a', assoc_sort)
for (_, e1), (_, e2) in product(ELEMS.items(), ELEMS.items()):
    if not e1.unbound and not e2.unbound:
        stmt = ForAll(
            [assoc_a],
            Iff(
                assoc_rel(e1.ref, assoc_a, e2.ref),
                Or(
                    *(
                        assoc_a == ASSOCS[e1_assoc_k].ref
                        for e1_assoc_k, e1_assoc_v in e1.associations.items()
                        if e2.id in e1_assoc_v
                    )
                )
            )
        )
        s.assert_and_track(stmt, f'AssocRel {e1.id} {e2.id}')

In [61]:
assert(s.check() == sat)

Since we're done with the setup, we need to be sure that the base model is satisfiable!